In [3]:
import pulp
import networkx as nx

# Data

from https://or.stackexchange.com/questions/5528/miller-tucker-zemlin-subtour-elimination-constraints-to-obtain-a-minimum-spannin

In [4]:
edges = [(1,2),(1,3),(1,4),
         (2,3),(2,5),(2,6),(2,7),
         (3,4),(3,7),
         (4,7),(4,10),
         (5,6),(5,8),(5,9),
         (6,7),(6,8),(7,8),(7,10),
         (8,9),(8,10),
         (9,10)]
weights = [4,1,4,
           5,9,9,7,
           3,9,
           10,18,
           2,4,6,
           8,2,
           9,8,
           3,9,
           9]
edges = dict(zip(edges,weights))

In [5]:
G=nx.Graph()
for (u,v) in edges:
    G.add_edge(u,v,cost=edges[(u,v)])

In [6]:
T = nx.minimum_spanning_tree(G,weight="cost")

In [7]:
sorted(T.edges(data=True))

[(1, 2, {'cost': 4}),
 (1, 3, {'cost': 1}),
 (2, 7, {'cost': 7}),
 (3, 4, {'cost': 3}),
 (5, 6, {'cost': 2}),
 (6, 7, {'cost': 8}),
 (6, 8, {'cost': 2}),
 (7, 10, {'cost': 8}),
 (8, 9, {'cost': 3})]

In [8]:
sum(G[u][v]["cost"] for (u,v) in sorted(T.edges()))

38

In [9]:
dual_edges = [("a","m"),("a","b"),("b","m"),
              ("a","c"),("e","m"),("e","f"),("f","c"),
             ("b","d"),("c","d"),
              ("g","d"),("g","m"),
              ("e","h"),("h","j"),("j","m"),
              ("f","i"),("i","h"),
              ("k","i"),("k","g"),
              ("j","l"),("l","k"),
              ("l","m")]
primal_edges = edges
intersection = dict(zip(dual_edges,primal_edges))

In [10]:
intersection

{('a', 'm'): (1, 2),
 ('a', 'b'): (1, 3),
 ('b', 'm'): (1, 4),
 ('a', 'c'): (2, 3),
 ('e', 'm'): (2, 5),
 ('e', 'f'): (2, 6),
 ('f', 'c'): (2, 7),
 ('b', 'd'): (3, 4),
 ('c', 'd'): (3, 7),
 ('g', 'd'): (4, 7),
 ('g', 'm'): (4, 10),
 ('e', 'h'): (5, 6),
 ('h', 'j'): (5, 8),
 ('j', 'm'): (5, 9),
 ('f', 'i'): (6, 7),
 ('i', 'h'): (6, 8),
 ('k', 'i'): (7, 8),
 ('k', 'g'): (7, 10),
 ('j', 'l'): (8, 9),
 ('l', 'k'): (8, 10),
 ('l', 'm'): (9, 10)}

In [11]:
G_dual = nx.Graph()
for (u,v) in dual_edges:
    G_dual.add_edge(u,v,primal_edge=intersection[(u,v)])

In [13]:
H=G.to_directed()
L=[]
for (u,v) in H.edges():
    if v==10:
        L.append((v,u))
H.remove_edges_from(L)

In [14]:
H_dual=G_dual.to_directed()
L=[]
for (u,v) in H.edges():
    if v=="m":
        L.append((v,u))
H_dual.remove_edges_from(L)

In [15]:
# déclaration du problème
prob=pulp.LpProblem("tree",pulp.LpMinimize)

# variables
x=pulp.LpVariable.dicts("x",H.edges(),cat=pulp.LpBinary)
y=pulp.LpVariable.dicts("y",H_dual.edges(),cat=pulp.LpBinary)

# objectif
prob += pulp.lpSum(x[(i,j)]*H[i][j]["cost"] for (i,j) in x)

# connectivity
for i in H.nodes():
    if i!=10:
        prob += pulp.lpSum(x[(i,j)] for j in H.nodes() if (i,j) in x)==1, "node_%s"%i
for i in H_dual.nodes():
    if i !="m":
        prob += pulp.lpSum(y[(i,j)] for j in H_dual.nodes() if (i,j) in y)==1, "node_%s"%i

# partition
for (i,j) in H_dual.edges():
    (k,l) = H_dual[i][j]["primal_edge"]
    if (k,l) in x and (l,k) in x and (i,j) in y and (j,i) in y:
        prob += y[(i,j)]+y[(j,i)] + x[(k,l)]+x[(l,k)] == 1

In [16]:
prob.solve()

1

In [17]:
pulp.value(prob.objective)

38.0